### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings, using one-vs-all SVM

In [54]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

In [2]:
from keras import backend as K

In [43]:
SEED=42

In [3]:
np.random.seed(seed=SEED)

In [4]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [5]:
preprocessed_texts = texts

In [6]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [8]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [9]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [10]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [11]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [12]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [13]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((40000, 1000), (40000, 1469))

In [14]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(GlobalAvgPool1D())
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')
# multi label needs a specific accuracy function


# loss doesn't get better after 5 epochs
model.fit(X_train, Y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=1)

Epoch 1/5
32000/32000 [==============================] - 3s - loss: 0.2795     
Epoch 2/5
32000/32000 [==============================] - 3s - loss: 0.0815     
Epoch 3/5
32000/32000 [==============================] - 3s - loss: 0.0429     
Epoch 4/5
32000/32000 [==============================] - 3s - loss: 0.0279     
Epoch 5/5
32000/32000 [==============================] - 3s - loss: 0.0207     
CPU times: user 21.7 s, sys: 2.22 s, total: 23.9 s
Wall time: 16 s


In [16]:
get_pooling_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [32]:
new_X_train = []
for raw_doc_features in X_train:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_train.append(layer_output[0].ravel())
new_X_train = np.vstack(new_X_train)

new_X_train.shape

(32000, 100)

In [34]:
new_X_val = []
for raw_doc_features in X_val:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_val.append(layer_output[0].ravel())
new_X_val = np.vstack([new_X_val])

new_X_val.shape

(8000, 100)

In [ ]:
# sample result for a couple of test cases
num_test_cases = 10
threshold=0.07

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1)).ravel()
              
#     predicted_label_indices = tags_helper.get_predicted_indices_by_threshold(predicted_tag_probabilities,threshold)
       
    predicted_label_indices = get_predicted_indices_by_tag_doc_fraction(
        tag_probabilities_index,
        predicted_tag_probabilities)
        
        
    actual_labels = lb.inverse_transform(actual_label_indices)
    
    predicted_labels = lb.inverse_transform(predicted_label_indices.reshape(1,-1))
   
    print(actual_labels)
    print(predicted_labels)
    print('\n')


In [57]:
clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.LinearSVC()),
]),n_jobs=-1)
parameters = [
    { 
          "estimator__clf__penalty": ["l1"],
          "estimator__clf__multi_class":["crammer_singer"],
          "estimator__clf__tol": [0.01,0.01],

    },
    { 
          "estimator__clf__penalty": ["l2"],
          "estimator__clf__dual":[False,True],
          "estimator__clf__multi_class":["crammer_singer","ovr"],
          "estimator__clf__tol": [0.01,0.001],

    },
]

In [ ]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(new_X_train,Y_train)
    
    Y_pred = clf.predict(new_X_val)
    
    current_score = f1_score(Y_val,Y_pred,average='micro')
    
    print("current_score was {} and the current grid was {}".format(current_score,g))
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

In [ ]:
best micro F1: 0.040004092490280335